In [2]:
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime
import time
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from math import floor, ceil
from pylab import rcParams
import re

%matplotlib inline
engine = sqlite3.connect('DB')

In [12]:
train_df = pd.read_csv('./train.csv')
train_df['Date'] = train_df['Date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))

In [46]:
weekIndex = []
indexTracker = 1
previousDept = 1
previousStore = 1
for i in range(len(train_df)):
    currStore = train_df.iloc[i].Store
    currDept = train_df.iloc[i].Dept
    if(currStore != previousStore or currDept != previousDept):
        indexTracker = 1
    if(indexTracker == 53):
        indexTracker = 1
        
    weekIndex.append(indexTracker)
    indexTracker += 1
    
    previousDept = currDept
    previousStore = currStore
    
train_df['weekIndex'] = weekIndex

In [47]:
train_df

,Store,Dept,Date,Weekly_Sales,IsHoliday,weekIndex
0,1,1,2010-02-05,24924.50,False,1
1,1,1,2010-02-12,46039.49,True,2
2,1,1,2010-02-19,41595.55,False,3
3,1,1,2010-02-26,19403.54,False,4
4,1,1,2010-03-05,21827.90,False,5
5,1,1,2010-03-12,21043.39,False,6
6,1,1,2010-03-19,22136.64,False,7
7,1,1,2010-03-26,26229.21,False,8
8,1,1,2010-04-02,57258.43,False,9
9,1,1,2010-04-09,42960.91,False,10


In [52]:
temp = train_df[420000:]
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(temp)
    
    
    dict[1][1][2]

        Store  Dept       Date  Weekly_Sales IsHoliday  weekIndex
420000     45    82 2012-08-10      13027.71     False         28
420001     45    82 2012-08-17      13622.04     False         29
420002     45    82 2012-08-24      12626.69     False         30
420003     45    82 2012-08-31      13872.96     False         31
420004     45    82 2012-09-07      13974.11      True         32
420005     45    82 2012-09-14      12737.58     False         33
420006     45    82 2012-09-21      11093.46     False         34
420007     45    82 2012-09-28      12044.60     False         35
420008     45    82 2012-10-05      13412.34     False         36
420009     45    82 2012-10-12      12558.39     False         37
420010     45    82 2012-10-19      13658.52     False         38
420011     45    82 2012-10-26      17360.33     False         39
420012     45    83 2010-02-05       1127.26     False          1
420013     45    83 2010-02-12        587.12      True          2
420014    

In [119]:
temp = train_df[(train_df['Store'] == 2) & (train_df['Dept'] == 4) & (train_df['weekIndex'] == 50)]
errorDf = train_df[['Store', 'Dept', 'weekIndex']].copy()
errorDf = errorDf.drop_duplicates()

In [120]:
temp

,Store,Dept,Date,Weekly_Sales,IsHoliday,weekIndex
10722,2,4,2011-01-14,44945.65,False,50
10774,2,4,2012-01-13,43649.89,False,50


In [92]:
total = 0
for i in range(len(temp.Weekly_Sales)):
    total += temp.Weekly_Sales.values[i]
print(total / len(temp.Weekly_Sales))
temp.Weekly_Sales.values

47072.450000000004


array([49046.81, 44543.25, 47627.29])

In [ ]:
#fullDict = {1: {1 : weekIndexSales, 2 : weekIndexSales}, 2: {4 : weekIndexSales}}
fullDict = {}


In [118]:
averagedSales = []
for i in range(1, 46):
    total = 0
    for j in range(1, 99):
        for k in range(1, 53):
            print(i, j, k)
#             week = train_df[(train_df['Store'] == i) & (train_df['Dept'] == j) & (train_df['weekIndex'] == k)]
            
#             for l in range(len(week.Weekly_Sales.values)):
#                 total += week.Weekly_Sales.values[l]
#             total = total / len(week)
#             averagedSales.append(total)

1 1 1
1 1 2
1 1 3
1 1 4
1 1 5
1 1 6
1 1 7
1 1 8
1 1 9
1 1 10
1 1 11
1 1 12
1 1 13
1 1 14
1 1 15
1 1 16
1 1 17
1 1 18
1 1 19
1 1 20
1 1 21
1 1 22
1 1 23
1 1 24
1 1 25
1 1 26
1 1 27
1 1 28
1 1 29
1 1 30
1 1 31
1 1 32
1 1 33
1 1 34
1 1 35
1 1 36
1 1 37
1 1 38
1 1 39
1 1 40
1 1 41
1 1 42
1 1 43
1 1 44
1 1 45
1 1 46
1 1 47
1 1 48
1 1 49
1 1 50
1 1 51
1 1 52
1 2 1
1 2 2
1 2 3
1 2 4
1 2 5
1 2 6
1 2 7
1 2 8
1 2 9
1 2 10
1 2 11
1 2 12
1 2 13
1 2 14
1 2 15
1 2 16
1 2 17
1 2 18
1 2 19
1 2 20
1 2 21
1 2 22
1 2 23
1 2 24
1 2 25
1 2 26
1 2 27
1 2 28
1 2 29
1 2 30
1 2 31
1 2 32
1 2 33
1 2 34
1 2 35
1 2 36
1 2 37
1 2 38
1 2 39
1 2 40
1 2 41
1 2 42
1 2 43
1 2 44
1 2 45
1 2 46
1 2 47
1 2 48
1 2 49
1 2 50
1 2 51
1 2 52
1 3 1
1 3 2
1 3 3
1 3 4
1 3 5
1 3 6
1 3 7
1 3 8
1 3 9
1 3 10
1 3 11
1 3 12
1 3 13
1 3 14
1 3 15
1 3 16
1 3 17
1 3 18
1 3 19
1 3 20
1 3 21
1 3 22
1 3 23
1 3 24
1 3 25
1 3 26
1 3 27
1 3 28
1 3 29
1 3 30
1 3 31
1 3 32
1 3 33
1 3 34
1 3 35
1 3 36
1 3 37
1 3 38
1 3 39
1 3 40
1 3 41
1 3 42
1 3 4

KeyboardInterrupt: 